In [15]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import fairlearn.metrics as flm
import sklearn.metrics as skm
from sklearn.metrics import confusion_matrix
from sklearn.exceptions import ConvergenceWarning

from disagg import naive_estimate, structured_regression

import warnings
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

# Definitions of performance metrics
def safe_ratio(num, den):
    if den==0:
        return np.nan
    else:
        return num / den

def false_negative_rate(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return safe_ratio(fn, fn+tp)

def false_positive_rate(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return safe_ratio(fp, fp+tn)

def precision_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return safe_ratio(tp, tp+fp)

def roc_auc_score(y_true, y_pred):
    if len(np.unique(y_true)) == 1:
        return np.nan
    else:
        return skm.roc_auc_score(y_true, y_pred)

METRICS = {
    'auc': roc_auc_score,
    'sel': flm.selection_rate,
    'fnr': false_negative_rate,
    'fpr': false_positive_rate,
    'acc': skm.accuracy_score,
    'ppv': precision_score
}

# Grid for cross-validation
ALPHA_LIST = [20.0, 10.0, 5.0, 2.0, 1.0, 0.5, 0.2, 0.1, 0.05, 0.02, 0.01, 0.005]

# Confidence interval values
CI_VALS = [50, 80, 90, 95]

# Fancy plotting of errors with clipping of values to the range of the metric
def plot_errors(*, ax, counts, vals, ci_lower=None, ci_upper=None, offset, marker='o', clip=None, label):
    skip_ci = False
    if (ci_lower is None) or (ci_upper is None):
        ci_lower = vals
        ci_upper = vals
        skip_ci = True
    if clip is not None:
        vals = vals.clip(clip[0], clip[1])
        ci_lower = ci_lower.clip(clip[0], clip[1])
        ci_upper = ci_upper.clip(clip[0], clip[1])
    sorted_index = counts.sort_values().index
    counts = counts.reindex(sorted_index)
    vals = vals.reindex(sorted_index)
    ci_lower = ci_lower.reindex(sorted_index).clip(upper=vals)
    ci_upper = ci_upper.reindex(sorted_index).clip(lower=vals)

    select = ~vals.isna()
    xs = np.arange(len(vals))
    yerr = np.array([vals[select]-ci_lower[select], ci_upper[select]-vals[select]])

    if skip_ci:
        ax.plot(xs[select]+offset, vals[select], marker=marker, ls='', label=label)
    else:
        ax.errorbar(xs[select]+offset, vals[select], yerr=yerr, marker=marker, ls='', label=label)
    
    ax.xaxis.set_tick_params(pad=12)
    ax.set_xticks(xs)
    ax.set_xticklabels(sorted_index, rotation=90, va='top', ha='center')
    [ymin, ymax] = ax.get_ylim()
    for i in range(len(vals)):
        ax.text(i, ymin-(ymax-ymin)*0.03, f"{counts[i]}", va='top', ha='center', c='r', fontsize=8)    


In [16]:
# Loading of diabetes evaluation data
def load_eval(*, metric, eval_id):
    
    # Load evaluation dataset
    df = pd.read_csv(f"diabetes/evaluation/eval{eval_id:02d}.csv")

    # Load ground-truth and set threshold for classification decisions
    df_truth = pd.read_csv('diabetes/evaluation/eval_all.csv')
    threshold = df_truth.S.quantile(0.8)

    # Generate ground-truth labels
    for df_i in [df, df_truth]:
        df_i['y'] = df_i.readmit_30_days.map({True: "y1", False: "y0"})
        df_i['y_true'] = df_i.readmit_30_days.astype(float)
        if metric == 'auc':
            df_i['y_pred'] = df_i.S 
        else:
            df_i['y_pred'] = 1.0*(df_i.S>threshold)

    # Generate features for regression
    cat1 = df[['race', 'gender', 'age', 'y']]
    cat2 = pd.DataFrame({
        'y_race': df['y'].str.cat(df['race'], sep='_'),
        'y_gender': df['y'].str.cat(df['gender'], sep='_'),
        'y_age': df['y'].str.cat(df['age'], sep='_'),
        'race_gender': df['race'].str.cat(df['gender'], sep='_'),
        'race_age': df['race'].str.cat(df['age'], sep='_'),
        'gender_age': df['gender'].str.cat(df['age'], sep='_')})
    cat3 = df[['grp_name']]
    static_cov = df[['number_inpatient', 'number_diagnoses', 'number_emergency', 'number_outpatient', 'chf']].astype(float)
    labels = df[['grp_name', 'y_true', 'y_pred']]
    static_cov = (static_cov - static_cov.mean()) / static_cov.std()

    dummies1 = pd.get_dummies(cat1)
    dummies2 = pd.get_dummies(cat2)
    dummies3 = pd.get_dummies(cat3)

    processed_df = pd.concat([labels, dummies1, dummies2, dummies3, static_cov], axis=1)

    # Consider three regression models called fa, fb, fc
    model_features = {
        'fa': list(dummies1) + list(dummies3),
        'fb': list(dummies1) + list(dummies3) + list(static_cov),
        'fc': list(dummies1) + list(dummies2) + list(dummies3) + list(static_cov),
    }
    return processed_df, model_features, df_truth

In [ ]:
# Example of a disaggregated evaluation
metric = 'fpr'
eval_id = 0

df, model_features, df_truth = load_eval(metric=metric, eval_id=eval_id)
naive = naive_estimate(
        df=df, metric=METRICS[metric], ci=CI_VALS)
lasso, _, _, _, _, _ = structured_regression(
        df=df, metric=METRICS[metric], feature_names=model_features['fb'], alpha_list=ALPHA_LIST, cv=True, ci=CI_VALS)
truth = naive_estimate(
        df=df_truth, metric=METRICS[metric])

fig, ax = plt.subplots(1, 1, figsize=[12,3])
counts = naive['count']
plot_errors(ax=ax, vals=naive.val, ci_lower=naive.ci95l, ci_upper=naive.ci95u, counts=counts, offset=-0.2, clip=[0,1], label='naive')
plot_errors(ax=ax, vals=lasso.val, ci_lower=lasso.ci95l_rlpr, ci_upper=lasso.ci95u_rlpr, counts=counts, offset=0, clip=[0,1], label='lasso')
plot_errors(ax=ax, vals=truth.val, counts=counts, offset=0.2, label='truth', marker='x')
ax.legend()
ax.set_ylabel(metric.upper())


In [ ]:
# Test implementation -- THIS TAKES A WHILE
for metric in METRICS:
    for eval_id in range(5):
        df, model_features, df_truth = load_eval(metric=metric, eval_id=eval_id)
        eval_str = f"{eval_id:02d}_{metric}"
        print(f"Evaluation dataset: {eval_str}")
        naive = naive_estimate(
            df=df, metric=METRICS[metric], ci=CI_VALS)
        lasso, _, _, _, _, _ = structured_regression(
            df=df, metric=METRICS[metric], feature_names=model_features['fb'], alpha_list=ALPHA_LIST, cv=True, ci=CI_VALS)
        naive_ref = pd.read_csv(f"diabetes/results/res{eval_str}_naive.csv", index_col='grp_name')
        lasso_ref = pd.read_csv(f"diabetes/results/res{eval_str}_lasso_fb.csv", index_col='grp_name')
        print(f"  naive difference: {(naive-naive_ref).abs().max().max()}")
        print(f"  lasso difference: {(lasso-lasso_ref).abs().max().max()}")